In [ ]:
{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[]},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","execution_count":null,"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"ClTtm5jmDlF9","outputId":"506e4b93-1d50-491a-8a8d-59c89f1327b0","executionInfo":{"status":"ok","timestamp":1744435080631,"user_tz":-330,"elapsed":11318,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"outputs":[{"output_type":"stream","name":"stdout","text":["Reading package lists... Done\n","Building dependency tree... Done\n","Reading state information... Done\n","libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).\n","0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.\n"]}],"source":["!apt-get install libopencv-dev"]},{"cell_type":"code","source":["import os\n","print(\"Number of CPU cores:\", os.cpu_count())"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"Gt6S-qVhIA23","outputId":"d52b6c69-f49f-4799-8951-6aa3d3b53e3e","executionInfo":{"status":"ok","timestamp":1744438774622,"user_tz":-330,"elapsed":7,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":8,"outputs":[{"output_type":"stream","name":"stdout","text":["Number of CPU cores: 2\n"]}]},{"cell_type":"code","source":["%%writefile sobel_openmp.cpp\n","\n","#include <iostream>\n","#include <cmath>\n","#include <opencv2/opencv.hpp>\n","#include <omp.h>\n","\n","extern \"C\" {\n","void sobel_filter(unsigned char* input, unsigned char* output, int width, int height) {\n","    int Gx[3][3] = { {-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1} };\n","    int Gy[3][3] = { {-1, -2, -1}, {0, 0, 0}, {1, 2, 1} };\n","\n","    #pragma omp parallel for collapse(2)\n","    for (int y = 1; y < height - 1; y++) {\n","        for (int x = 1; x < width - 1; x++) {\n","            int sumX = 0, sumY = 0;\n","\n","            for (int j = -1; j <= 1; j++) {\n","                for (int i = -1; i <= 1; i++) {\n","                    int pixel = input[(y + j) * width + (x + i)];\n","                    sumX += pixel * Gx[j + 1][i + 1];\n","                    sumY += pixel * Gy[j + 1][i + 1];\n","                }\n","            }\n","\n","            int magnitude = sqrt(sumX * sumX + sumY * sumY);\n","            output[y * width + x] = (magnitude > 255) ? 255 : magnitude;\n","        }\n","    }\n","}\n","}"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"iwOXjgeuKJkk","outputId":"ff33e8de-9a8c-433b-d5a2-8be94882f412","executionInfo":{"status":"ok","timestamp":1744438786820,"user_tz":-330,"elapsed":342,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":9,"outputs":[{"output_type":"stream","name":"stdout","text":["Overwriting sobel_openmp.cpp\n"]}]},{"cell_type":"code","source":["!g++ -fopenmp -shared -o sobel_openmp.so -fPIC `pkg-config --cflags --libs opencv4` sobel_openmp.cpp"],"metadata":{"id":"iVu5znu0KQAw","executionInfo":{"status":"ok","timestamp":1744438794443,"user_tz":-330,"elapsed":3407,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":10,"outputs":[]},{"cell_type":"code","source":["import ctypes\n","import numpy as np\n","import cv2\n","import matplotlib.pyplot as plt\n","\n","# Load OpenMP shared library\n","sobel_lib = ctypes.CDLL(\"./sobel_openmp.so\")\n","\n","# Define function signature: void sobel_filter(unsigned char*, unsigned char*, int, int)\n","sobel_lib.sobel_filter.argtypes = [ctypes.POINTER(ctypes.c_ubyte),\n","                                   ctypes.POINTER(ctypes.c_ubyte),\n","                                   ctypes.c_int, ctypes.c_int]\n","\n","# Load image\n","image_path = \"/content/sample_data/Lenna_test_image.png\"  # Change this\n","image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)\n","\n","# Get image dimensions\n","height, width = image.shape\n","\n","# Convert image to 1D array\n","input_image = np.ascontiguousarray(image, dtype=np.uint8)\n","output_image = np.zeros_like(input_image, dtype=np.uint8)\n","\n","# Convert numpy arrays to ctypes pointers\n","input_ptr = input_image.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte))\n","output_ptr = output_image.ctypes.data_as(ctypes.POINTER(ctypes.c_ubyte))\n","\n","# Call the OpenMP Sobel filter function\n","%time sobel_lib.sobel_filter(input_ptr, output_ptr, width, height)\n","\n","# Reshape back to 2D image\n","edges = output_image.reshape((height, width))\n","\n","# Show original & edge-detected image\n","plt.figure(figsize=(10, 5))\n","plt.subplot(1, 2, 1)\n","plt.imshow(image, cmap=\"gray\")\n","plt.title(\"Original Grayscale Image\")\n","plt.axis(\"off\")\n","\n","plt.subplot(1, 2, 2)\n","plt.imshow(edges, cmap=\"gray\")\n","plt.title(\"Edge Detected Image (Sobel - OpenMP C++)\")\n","plt.axis(\"off\")\n","\n","plt.show()\n","\n","# Save the output image\n","cv2.imwrite(\"sobel_openmp_output.png\", edges)"],"metadata":{"colab":{"base_uri":"https://localhost:8080/","height":463},"id":"ri_j6Ql5K4RW","outputId":"ba9ef18b-d636-4693-ca23-010c5e286b63","executionInfo":{"status":"ok","timestamp":1744438813760,"user_tz":-330,"elapsed":1548,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":11,"outputs":[{"output_type":"stream","name":"stdout","text":["CPU times: user 31.7 ms, sys: 0 ns, total: 31.7 ms\n","Wall time: 16.4 ms\n"]},{"output_type":"display_data","data":{"text/plain":["<Figure size 1000x500 with 2 Axes>"],"image/png":"...\n"},"metadata":{}},{"output_type":"execute_result","data":{"text/plain":["True"]},"metadata":{},"execution_count":11}]}]}